In [1]:
import pandas as pd

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:

answers = pd.read_csv("../english_texts.csv")

In [4]:
# Selecting only 'TEXT' and 'cEXT' columns and renaming them
df_ext = answers[['TEXT', 'cEXT']].rename(columns={'TEXT': 'text', 'cEXT': 'labels'})

# Reordering columns as per your requirement
df_ext = df_ext[['text', 'labels']]

In [5]:
df_ext

,text,labels
0,"Well, right now I just woke up from a mid-day ...",0
1,"Well, here we go with the stream of consciousn...",0
2,An open keyboard and buttons to push. The thin...,0
3,I can't believe it! It's really happening! M...,1
4,"Well, here I go with the good old stream of co...",1
...,...,...
2958,I am motivated on a day to day basis by the ne...,1
2959,My son is the biggest part of my life and with...,1
2960,My kids and grandkids are what keeps me motiva...,1
2961,My biggest drive is to earn money so I can ret...,0


In [7]:
from sklearn.model_selection import train_test_split

# Assuming df_ext is your DataFrame with 'text' and 'labels' columns

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df_ext['text'], df_ext['labels'], random_state=42, test_size=0.4
)

# Creating a list of dictionaries for the test set
test_data = [{'label': int(label), 'text': text} for label, text in zip(y_test, X_test)]
train_data = [{'label': int(label), 'text': text} for label, text in zip(y_train, X_train)]

# Creating the final data structure
extroversion = {
    'train': train_data,
    'test': test_data
}


In [8]:
df_ext['text_length'] = df_ext['text'].apply(len)
average_length = df_ext['text_length'].mean()
median_length = df_ext['text_length'].median()

average_length, median_length

(3161.278434019575, 2992.0)

In [9]:
extroversion['train'][0]['text']

"I know that I have assignments to take care of but I do not want to complete them. I know it is incredibly important for me to perform these tasks. they can make or break my future. yet, I still never seem to be able to motivate myself. it is really frustrating sometimes. But that  is were the apathy sets in again and I will not take action. It seems like a cycle that I know at my house. My dad had behavior problems, to put them modestly, I could see how it would pain him, he did want to change, but it was as if he never followed through. I do not blame  him, I just see a pattern that is similar to mine. But at least I am working now. I still have German 506 to complete that is were I am most suffering. I need to review, and starting to catch up before I wind up failing. It would b e the realization of possibly my worst nightmare. Me being my only downfall, or rather me causing my own downfall. At some point I have to break this cycle but sometimes I don't think I will. But I must pre

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:


tokens = tokenizer.encode_plus(extroversion['train'][0]['text'], add_special_tokens=False,
                               return_tensors='pt')

print(len(tokens['input_ids'][0]))
tokens



893


{'input_ids': tensor([[ 1045,  2113,  2008,  1045,  2031, 14799,  2000,  2202,  2729,  1997,
          2021,  1045,  2079,  2025,  2215,  2000,  3143,  2068,  1012,  1045,
          2113,  2009,  2003, 11757,  2590,  2005,  2033,  2000,  4685,  2122,
          8518,  1012,  2027,  2064,  2191,  2030,  3338,  2026,  2925,  1012,
          2664,  1010,  1045,  2145,  2196,  4025,  2000,  2022,  2583,  2000,
          9587, 29068,  3686,  2870,  1012,  2009,  2003,  2428, 25198,  2823,
          1012,  2021,  2008,  2003,  2020,  1996,  9706, 17308,  4520,  1999,
          2153,  1998,  1045,  2097,  2025,  2202,  2895,  1012,  2009,  3849,
          2066,  1037,  5402,  2008,  1045,  2113,  2012,  2026,  2160,  1012,
          2026,  3611,  2018,  5248,  3471,  1010,  2000,  2404,  2068, 10754,
          2135,  1010,  1045,  2071,  2156,  2129,  2009,  2052,  3255,  2032,
          1010,  2002,  2106,  2215,  2000,  2689,  1010,  2021,  2009,  2001,
          2004,  2065,  2002,  2196,  

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [11]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(pd.DataFrame(extroversion['train']))
test_dataset = Dataset.from_pandas(pd.DataFrame(extroversion['test']))

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1777 [00:00<?, ? examples/s]

Map:   0%|          | 0/1186 [00:00<?, ? examples/s]

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    f1score = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1score": f1score}

In [14]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "allenai/longformer-base-4096", num_labels=2, id2label=id2label, label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
output_dir = '../checkpoints/transformer_huggingface'


In [17]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a LongformerTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 9.19 GB, other allocations: 8.83 GB, max allowed: 18.13 GB). Tried to allocate 144.28 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).


Hyperparamater tuning

In [ ]:
# output_dir_hyper = '/content/drive/MyDrive/Levy/transformer_huggingface_hyper'


In [ ]:
# %pip install wandb

In [ ]:
# def wandb_hp_space(trial):
#     return {
#         "method": "bayes",
#         "metric": {"name": "objective", "goal": "minimize"},
#         "parameters": {
#             "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 5e-5},
#             "per_device_train_batch_size": {"values": [8, 16]},
#             "num_train_epochs": {"values": [5, 10, 15]},
#         },
#     }

In [ ]:
# def model_init():
#     return AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
# )

In [ ]:
# training_args_hyper = TrainingArguments(
#     output_dir=output_dir_hyper,
#     per_device_eval_batch_size=16,
#     per_device_train_batch_size=16,
#     num_train_epochs=5,
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=False,
#     logging_steps=100,
# )

In [ ]:
# trainer = Trainer(
#     model=None,
#     model_init=model_init,
#     args=training_args_hyper,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_test,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     data_collator=data_collator,
# )

In [ ]:
# best_trial = trainer.hyperparameter_search(
#     direction="minimize",
#     backend="wandb",
#     hp_space=wandb_hp_space,
#     n_trials=15,
# )